In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
admis = pd.read_csv('../Data Files/NSS_Hospital_Inpatient_Admissions.csv',
                    infer_datetime_format=True, parse_dates=[8,9],
                   dtype= {6:'str',12:'str',14:'str',15:'str'})
visits = pd.read_csv('../Data Files/NSS_PrimaryCare_Office_Visits.csv',
                     infer_datetime_format=True, parse_dates=[12],
                    dtype={8:'str',9:'str',10:'str',11:'str'})
mem = pd.read_csv('../Data Files/NSS_GreatCare_Members.csv',
                  infer_datetime_format=True, parse_dates=[5,61,62],
                 dtype= {6:'str',10:'str',11:'str',13:'str'})
npi = pd.read_csv('../output/npi.csv')
print(admis.shape)
print(visits.shape)
print(mem.shape)

C:\Users\pfbab\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(6213, 23)
(231373, 18)
(61519, 63)


In [ ]:
pcp_visit = 0
readmit = 0
#pull patient numbers that have more then one admission to loop through
for patnum in admis['Patient ID'].drop_duplicates():
    window = pd.to_datetime(0) #set window as a datetime to make first loop work
    #create dataframe for each patient number keeping only the final discharge per Start Date
    patient = admis[admis['Patient ID']==patnum]
    patient = patient.sort_values(['Encounter Start Date','Encounter End Date']).drop_duplicates(subset=['Patient ID','Encounter End Date'], keep='first')
    patient = patient.sort_values('Encounter End Date').drop_duplicates(subset=['Patient ID','Encounter Start Date'], keep='last')
    vpatient = visits[visits['Patient ID']==patnum].sort_values('Encounter Date')
    
    for rnum in range(0,len(patient)):
        if window < patient.iloc[rnum,8] :
            #test all admissions after rnum
            trigger = patient.iloc[rnum+1:,8]<=(patient.iloc[rnum,9] + pd.DateOffset(days=30)) #column 8=start, 9=end
            #add one if there are Trues
            readmit = readmit + int(trigger.sum() > 0)
            #save 30 day window date to remove dupicate dates
            window = (patient.iloc[rnum,9] + pd.DateOffset(days=30))
            if trigger.sum() > 0:
                admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'index_admis'] = True
                admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'readmission'] = True
            else:
                admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'index_admis'] = True
        else:
            admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'index_admis'] = False
        trigger7 = vpatient[vpatient['Encounter Date']>=patient.iloc[rnum,9]]['Encounter Date']  <=  (patient.iloc[rnum,9] + pd.DateOffset(days=7))
        pcp_visit += int(trigger7.sum() > 0)
        if trigger7.sum() > 0:
            admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'pcp_visit'] = True
        else:
            admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'pcp_visit'] = False

In [ ]:
admis['index_admis'].value_counts()

In [ ]:
(admis['index_admis'].isna()).sum()

In [ ]:
len(admis)

In [ ]:
readmit

In [ ]:
pcp_visit

In [ ]:
#question1
q1num = len(admis[(admis['index_admis']==True)&(admis['readmission']==True)])
q1denom = admis['index_admis'].value_counts()[True]
print('numerator: ' + str(q1num))
print('denominator: ' + str(q1denom))
print(q1num/q1denom)


In [ ]:
#question2
q2num = admis['pcp_visit'].value_counts()[True]
q2denom = admis['index_admis'].value_counts().sum()
q2num

In [ ]:
admis['pcp_visit'].value_counts()

In [ ]:
visittimeline[rnum]

In [ ]:
pcp_visit